In [0]:
#sdasdasd
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional , CuDNNLSTM
import math
from sklearn.metrics import mean_squared_error
!pip install pandas-datareader
from pandas_datareader import data as pdr
!pip install fix-yahoo-finance
import fix_yahoo_finance as yf


Using TensorFlow backend.


In [0]:
share = 'SPY'
yf
dataset = yf.download(share,"2003-3-14") #, start="2017-01-01"
dataset = dataset.drop(columns=['Open','High', 'Low','Adj Close','Volume'])
dataset.head()

[*********************100%***********************]  1 of 1 downloaded


,Close
Date,
2003-03-14,84.129997
2003-03-17,86.779999
2003-03-18,87.290001
2003-03-19,87.959999
2003-03-20,88.150002


In [0]:
dataset['MA'] = dataset.rolling(window=5).mean()
dataset


,Close,MA
Date,,
2003-03-14,84.129997,NaN
2003-03-17,86.779999,NaN
2003-03-18,87.290001,NaN
2003-03-19,87.959999,NaN
2003-03-20,88.150002,86.862000
2003-03-21,89.669998,87.970000
2003-03-24,86.690002,87.952000
2003-03-25,87.519997,87.998000
2003-03-26,87.080002,87.822000


In [0]:
print(dataset.head(1).Close)
print(len(dataset))

Date
2003-03-14    84.129997
Name: Close, dtype: float64
4116


In [0]:
for i, row in dataset.iterrows():
    print(i)
    print(row.MA.round(2))

2003-03-14 00:00:00
nan
2003-03-17 00:00:00
nan
2003-03-18 00:00:00
nan
2003-03-19 00:00:00
nan
2003-03-20 00:00:00
86.86
2003-03-21 00:00:00
87.97
2003-03-24 00:00:00
87.95
2003-03-25 00:00:00
88.0
2003-03-26 00:00:00
87.82
2003-03-27 00:00:00
87.62
2003-03-28 00:00:00
87.03
2003-03-31 00:00:00
86.64
2003-04-01 00:00:00
86.34
2003-04-02 00:00:00
86.55
2003-04-03 00:00:00
86.66
2003-04-04 00:00:00
86.96
2003-04-07 00:00:00
87.63
2003-04-08 00:00:00
88.06
2003-04-09 00:00:00
87.84
2003-04-10 00:00:00
87.8
2003-04-11 00:00:00
87.59
2003-04-14 00:00:00
87.77
2003-04-15 00:00:00
88.08
2003-04-16 00:00:00
88.33
2003-04-17 00:00:00
88.74
2003-04-21 00:00:00
89.24
2003-04-22 00:00:00
89.72
2003-04-23 00:00:00
90.2
2003-04-24 00:00:00
90.82
2003-04-25 00:00:00
90.95
2003-04-28 00:00:00
91.38
2003-04-29 00:00:00
91.53
2003-04-30 00:00:00
91.48
2003-05-01 00:00:00
91.59
2003-05-02 00:00:00
92.18
2003-05-05 00:00:00
92.43
2003-05-06 00:00:00
92.79
2003-05-07 00:00:00
93.09
2003-05-08 00:00:00
93.

In [0]:
df2 = dataset.iloc[::30, :]
df2.head()

,Close,MA
Date,,
2003-03-14,84.129997,NaN
2003-04-28,91.790001,91.380000
2003-06-10,99.250000,99.114002
2003-07-23,99.239998,98.939999
2003-09-04,103.410004,102.354002


In [0]:
invPerMonth = 1000
total =0
rem=0 
shares=0 

for i, row in df2.iterrows():
   # print(str(i) +" | "+ str(row.Close.round(2)))
    numShares = math.floor((invPerMonth)/row.Close.round(2))
    shares+=numShares
   #  rem = invPerMonth -(row.Close.round(2) * numShares)
   # print("numShares:" , numShares)
   # print("shares:" , shares)
   # print("rem:" , rem)

print("num of shares" , shares)
print("num of shares * last closing ",int(dataset.tail(1).Close*shares)) #dataset.tail, not df2.tail


num of shares 885
num of shares * last closing  262995




 192,000 saved in total. apx 879 shares worth 290.16 each, for a total of $255050.64 for a ROI of 255050.64/192000 = 132.84%
 
 with remainder being invested, we should get the expected  %236 (on the closing of 5/29/2019)






In [0]:
invPerMonth = 1000
total =0
shares=0 

for i, row in df2.iterrows():
   # print("\n" + str(i) +" | "+ str(row.Close.round(2)))
    numShares = math.floor((invPerMonth)/row.Close.round(2))
    shares+=numShares
    invPerMonth -= (row.Close.round(2) * numShares)
    
    invPerMonth += 1000
   # print("numShares:" , numShares)
   # print("shares:" , shares)
   # print("invPerMonth:" , invPerMonth)

print("invPerMonth: ",invPerMonth)
print("num of shares" , shares)
print("num of shares * last closing ",int(dataset.tail(1).Close*shares)) #dataset.tail, not df2.tail


invPerMonth:  1283.819999999998
num of shares 958
num of shares * last closing  284688


192,000 saved in total. apx 879 shares worth 290.16 each, for a total of $255050.64 for a ROI of 255050.64/192000 = 132.84%

with remainder being invested, we should get the expected %236 (on the closing of 5/29/2019)


264913/120000 = 220.76%


In [0]:
dataset['MA'] = dataset.rolling(window=30).mean()


invPerMonth = 1000
total =0
shares=0 

for i, row in df2.iterrows():
   # print("\n" + str(i) +" | "+ str(row.Close.round(2)))
    if((row.Close.round(2) <= row.MA.round(2)) and not math.isnan(row.MA.round(2))):
      numShares = math.floor((invPerMonth)/row.Close.round(2))
      shares+=numShares
      invPerMonth -= (row.Close.round(2) * numShares)
      #print("\n close: " + str(i) +" | "+ str(row.Close.round(2)))
      #print(" MA:    "+str(i) +" | "+ str(row.MA.round(2)))
    invPerMonth += 1000
   # print("numShares:" , numShares)
   # print("shares:" , shares)
   # print("invPerMonth:" , invPerMonth)

print("invPerMonth: ",invPerMonth)
print("num of shares" , shares)
print("num of shares * last closing = ",int(dataset.tail(1).Close*shares)) #dataset.tail, not df2.tail
print("total: ", (int(dataset.tail(1).Close*shares)+invPerMonth))

invPerMonth:  2145.5399999999995
num of shares 942
num of shares * last closing =  279934
total:  282079.54


above is the stratergy is which be 

In [0]:
dataset['MA'] = dataset.rolling(window=14).mean()


invPerMonth = 1000
total =0
shares=0 

for i, row in df2.iterrows():
   # print("\n" + str(i) +" | "+ str(row.Close.round(2)))
    if((row.Close.round(2) <= row.MA.round(2)) and not math.isnan(row.MA.round(2))):
      numShares = math.floor((invPerMonth)/row.Close.round(2))
      shares+=numShares
      invPerMonth -= (row.Close.round(2) * numShares)
      #print("\n close: " + str(i) +" | "+ str(row.Close.round(2)))
      #print(" MA:    "+str(i) +" | "+ str(row.MA.round(2)))
    invPerMonth += 1000
   # print("numShares:" , numShares)
   # print("shares:" , shares)
   # print("invPerMonth:" , invPerMonth)

print("invPerMonth: ",invPerMonth)
print("num of shares" , shares)
print("num of shares * last closing = ",int(dataset.tail(1).Close*shares)) #dataset.tail, not df2.tail
print("total: ", (int(dataset.tail(1).Close*shares)+invPerMonth))

invPerMonth:  2112.469999999999
num of shares 940
num of shares * last closing =  279339
total:  281451.47


In [0]:
dataset['MA'] = dataset.rolling(window=7).mean()


invPerMonth = 1000
total =0
shares=0 

for i, row in df2.iterrows():
   # print("\n" + str(i) +" | "+ str(row.Close.round(2)))
    if((row.Close.round(2) <= row.MA.round(2)) and not math.isnan(row.MA.round(2))):
      numShares = math.floor((invPerMonth)/row.Close.round(2))
      shares+=numShares
      invPerMonth -= (row.Close.round(2) * numShares)
      #print("\n close: " + str(i) +" | "+ str(row.Close.round(2)))
      #print(" MA:    "+str(i) +" | "+ str(row.MA.round(2)))
    invPerMonth += 1000
   # print("numShares:" , numShares)
   # print("shares:" , shares)
   # print("invPerMonth:" , invPerMonth)

print("invPerMonth: ",invPerMonth)
print("num of shares" , shares)
print("num of shares * last closing = ",int(dataset.tail(1).Close*shares)) #dataset.tail, not df2.tail
print("total: ", (int(dataset.tail(1).Close*shares)+invPerMonth))

invPerMonth:  2255.170000000001
num of shares 943
num of shares * last closing =  280231
total:  282486.17


In [0]:
dataset['MA'] = dataset.rolling(window=1).mean()


invPerMonth = 1000
total =0
shares=0 

for i, row in df2.iterrows():
   # print("\n" + str(i) +" | "+ str(row.Close.round(2)))
    if((row.Close.round(2) <= row.MA.round(2)) and not math.isnan(row.MA.round(2))):
      numShares = math.floor((invPerMonth)/row.Close.round(2))
      shares+=numShares
      invPerMonth -= (row.Close.round(2) * numShares)
      #print("\n close: " + str(i) +" | "+ str(row.Close.round(2)))
      #print(" MA:    "+str(i) +" | "+ str(row.MA.round(2)))
    invPerMonth += 1000
   # print("numShares:" , numShares)
   # print("shares:" , shares)
   # print("invPerMonth:" , invPerMonth)

print("invPerMonth: ",invPerMonth)
print("num of shares" , shares)
print("num of shares * last closing = ",int(dataset.tail(1).Close*shares)) #dataset.tail, not df2.tail
print("total: ", (int(dataset.tail(1).Close*shares)+invPerMonth))

invPerMonth:  1283.819999999998
num of shares 958
num of shares * last closing =  284688
total:  285971.82


if we buy only higher then the moving average



In [0]:
dataset['MA'] = dataset.rolling(window=20).mean()


invPerMonth = 1000
total =0
shares=0 

for i, row in df2.iterrows():
   # print("\n" + str(i) +" | "+ str(row.Close.round(2)))
    if((row.Close.round(2) >= row.MA.round(2)) and not math.isnan(row.MA.round(2))):
      numShares = math.floor((invPerMonth)/row.Close.round(2))
      shares+=numShares
      invPerMonth -= (row.Close.round(2) * numShares)
      #print("\n close: " + str(i) +" | "+ str(row.Close.round(2)))
      #print(" MA:    "+str(i) +" | "+ str(row.MA.round(2)))
    invPerMonth += 1000
   # print("numShares:" , numShares)
   # print("shares:" , shares)
   # print("invPerMonth:" , invPerMonth)

print("invPerMonth: ",invPerMonth)
print("num of shares" , shares)
print("num of shares * last closing = ",int(dataset.tail(1).Close*shares)) #dataset.tail, not df2.tail
print("total: ", (int(dataset.tail(1).Close*shares)+invPerMonth))

invPerMonth:  1165.6800000000003
num of shares 959
num of shares * last closing =  284986
total:  286151.68


In [0]:
dataset['MA'] = dataset.rolling(window=90).mean()


invPerMonth = 1000
total =0
shares=0 

for i, row in df2.iterrows():
   # print("\n" + str(i) +" | "+ str(row.Close.round(2)))
    if((row.Close.round(2) >= row.MA.round(2)) and not math.isnan(row.MA.round(2))):
      numShares = math.floor((invPerMonth)/row.Close.round(2))
      shares+=numShares
      invPerMonth -= (row.Close.round(2) * numShares)
      #print("\n close: " + str(i) +" | "+ str(row.Close.round(2)))
      #print(" MA:    "+str(i) +" | "+ str(row.MA.round(2)))
    invPerMonth += 1000
   # print("numShares:" , numShares)
   # print("shares:" , shares)
   # print("invPerMonth:" , invPerMonth)

print("invPerMonth: ",invPerMonth)
print("num of shares" , shares)
print("num of shares * last closing = ",int(dataset.tail(1).Close*shares)) #dataset.tail, not df2.tail
print("total: ", (int(dataset.tail(1).Close*shares)+invPerMonth))

invPerMonth:  1119.6100000000015
num of shares 959
num of shares * last closing =  284986
total:  286105.61


In [0]:
dataset['MA'] = dataset.rolling(window=30).mean()


invPerMonth = 1000
total =0
shares=0 

for i, row in df2.iterrows():
   # print("\n" + str(i) +" | "+ str(row.Close.round(2)))
    if((row.Close.round(2) >= row.MA.round(2)) and not math.isnan(row.MA.round(2))):
      numShares = math.floor((invPerMonth)/row.Close.round(2))
      shares+=numShares
      invPerMonth -= (row.Close.round(2) * numShares)
      #print("\n close: " + str(i) +" | "+ str(row.Close.round(2)))
      #print(" MA:    "+str(i) +" | "+ str(row.MA.round(2)))
    invPerMonth += 1000
   # print("numShares:" , numShares)
   # print("shares:" , shares)
   # print("invPerMonth:" , invPerMonth)

print("invPerMonth: ",invPerMonth)
print("num of shares" , shares)
print("num of shares * last closing = ",int(dataset.tail(1).Close*shares)) #dataset.tail, not df2.tail
print("total: ", (int(dataset.tail(1).Close*shares)+invPerMonth))

invPerMonth:  1241.7800000000007
num of shares 959
num of shares * last closing =  284986
total:  286227.78


In [0]:
dataset['MA'] = dataset.rolling(window=20).mean()


invPerMonth = 1000
total =0
shares=0 

for i, row in df2.iterrows():
   # print("\n" + str(i) +" | "+ str(row.Close.round(2)))
    if((row.Close.round(2) >= row.MA.round(2)) and not math.isnan(row.MA.round(2))):
      numShares = math.floor((invPerMonth)/row.Close.round(2))
      shares+=numShares
      invPerMonth -= (row.Close.round(2) * numShares)
      #print("\n close: " + str(i) +" | "+ str(row.Close.round(2)))
      #print(" MA:    "+str(i) +" | "+ str(row.MA.round(2)))
    invPerMonth += 1000
   # print("numShares:" , numShares)
   # print("shares:" , shares)
   # print("invPerMonth:" , invPerMonth)

print("invPerMonth: ",invPerMonth)
print("num of shares" , shares)
print("num of shares * last closing = ",int(dataset.tail(1).Close*shares)) #dataset.tail, not df2.tail
print("total: ", (int(dataset.tail(1).Close*shares)+invPerMonth))

invPerMonth:  1165.6800000000003
num of shares 959
num of shares * last closing =  284986
total:  286151.68


In [0]:
dataset['MA'] = dataset.rolling(window=7).mean()


invPerMonth = 1000
total =0
shares=0 

for i, row in df2.iterrows():
   # print("\n" + str(i) +" | "+ str(row.Close.round(2)))
    if((row.Close.round(2) >= row.MA.round(2)) and not math.isnan(row.MA.round(2))):
      numShares = math.floor((invPerMonth)/row.Close.round(2))
      shares+=numShares
      invPerMonth -= (row.Close.round(2) * numShares)
      #print("\n close: " + str(i) +" | "+ str(row.Close.round(2)))
      #print(" MA:    "+str(i) +" | "+ str(row.MA.round(2)))
    invPerMonth += 1000
   # print("numShares:" , numShares)
   # print("shares:" , shares)
   # print("invPerMonth:" , invPerMonth)

print("invPerMonth: ",invPerMonth)
print("num of shares" , shares)
print("num of shares * last closing = ",int(dataset.tail(1).Close*shares)) #dataset.tail, not df2.tail
print("total: ", (int(dataset.tail(1).Close*shares)+invPerMonth))

invPerMonth:  1071.780000000001
num of shares 961
num of shares * last closing =  285580
total:  286651.78
